# RL-mallin kouluttaminen tasapainottamaan Cartpole

Tämä muistikirja on osa [AI for Beginners -opetussuunnitelmaa](http://aka.ms/ai-beginners). Se on saanut inspiraationsa [virallisesta PyTorch-oppaasta](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) ja [tästä Cartpole PyTorch -toteutuksesta](https://github.com/yc930401/Actor-Critic-pytorch).

Tässä esimerkissä käytämme RL:ää (vahvistusoppimista) kouluttaaksemme mallin tasapainottamaan tankoa vaunussa, joka voi liikkua vasemmalle ja oikealle vaakasuoralla tasolla. Käytämme [OpenAI Gym](https://www.gymlibrary.ml/) -ympäristöä simuloimaan tankoa.

> **Huomio**: Voit suorittaa tämän oppitunnin koodin paikallisesti (esim. Visual Studio Codessa), jolloin simulaatio avautuu uuteen ikkunaan. Jos suoritat koodin verkossa, sinun saattaa olla tarpeen tehdä joitakin muutoksia koodiin, kuten kuvattu [tässä](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

Aloitamme varmistamalla, että Gym on asennettu:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Nyt luodaan CartPole-ympäristö ja katsotaan, miten sitä käytetään. Ympäristöllä on seuraavat ominaisuudet:

* **Toimintotila** on joukko mahdollisia toimintoja, joita voimme suorittaa jokaisessa simulaation vaiheessa  
* **Havaintotila** on tila, jossa voimme tehdä havaintoja


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Katsotaan, miten simulaatio toimii. Seuraava silmukka suorittaa simulaation, kunnes `env.step` palauttaa lopetuslipun `done`. Valitsemme toiminnot satunnaisesti käyttämällä `env.action_space.sample()`, mikä tarkoittaa, että koe epäonnistuu todennäköisesti hyvin nopeasti (CartPole-ympäristö päättyy, kun CartPolen nopeus, sijainti tai kulma ylittävät tietyt rajat).

> Simulaatio avautuu uuteen ikkunaan. Voit suorittaa koodin useita kertoja ja nähdä, miten se käyttäytyy.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Voit huomata, että havainnot sisältävät neljä lukua. Ne ovat:
- Kärryn sijainti
- Kärryn nopeus
- Tangon kulma
- Tangon pyörimisnopeus

`rew` on palkkio, jonka saamme jokaisella askeleella. CartPole-ympäristössä saat yhden pisteen jokaisesta simulaatioaskeleesta, ja tavoitteena on maksimoida kokonaispalkkio, eli aika, jonka CartPole pystyy tasapainottamaan kaatumatta.

Vahvistusoppimisen aikana tavoitteemme on kouluttaa **politiikka** $\pi$, joka jokaisessa tilassa $s$ kertoo meille, mikä toiminto $a$ tulisi valita, eli käytännössä $a = \pi(s)$.

Jos haluat todennäköisyyspohjaisen ratkaisun, voit ajatella politiikan palauttavan joukon todennäköisyyksiä jokaiselle toiminnolle, eli $\pi(a|s)$ tarkoittaisi todennäköisyyttä, että meidän tulisi valita toiminto $a$ tilassa $s$.

## Politiikkagradienttimenetelmä

Yksinkertaisimmassa RL-algoritmissa, jota kutsutaan **politiikkagradientiksi**, koulutamme neuroverkon ennustamaan seuraavan toiminnon.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Aiomme kouluttaa verkkoa suorittamalla useita kokeita ja päivittämällä verkkomme jokaisen ajon jälkeen. Määritellään funktio, joka suorittaa kokeen ja palauttaa tulokset (niin sanottu **jälki**) - kaikki tilat, toiminnot (ja niiden suositellut todennäköisyydet) sekä palkkiot:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Voit ajaa yhden jakson kouluttamattomalla verkolla ja huomata, että kokonaispalkkio (eli jakson pituus) on erittäin alhainen:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

Yksi politiikkagradienttialgoritmin hankalista puolista on käyttää **diskontattuja palkintoja**. Ajatuksena on, että laskemme pelin jokaisessa vaiheessa kokonaispalkintojen vektorin, ja tämän prosessin aikana diskonttaamme aikaiset palkinnot käyttäen jotakin kerrointa $gamma$. Normalisoimme myös tuloksena olevan vektorin, koska käytämme sitä painona vaikuttamaan koulutukseemme:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Nyt aloitetaan varsinainen harjoittelu! Suoritamme 300 jaksoa, ja jokaisessa jaksossa teemme seuraavat vaiheet:

1. Suorita koe ja kerää jälki
1. Laske ero (`gradients`) toteutettujen toimien ja ennustettujen todennäköisyyksien välillä. Mitä pienempi ero on, sitä varmemmin olemme tehneet oikean toiminnan.
1. Laske diskontatut palkkiot ja kerro gradientit diskontatuilla palkkioilla - tämä varmistaa, että korkeampia palkkioita sisältävät askeleet vaikuttavat lopputulokseen enemmän kuin matalampia palkkioita sisältävät.
1. Odotetut tavoitetoimet neuroverkkoamme varten otetaan osittain ennustetuista todennäköisyyksistä kokeen aikana ja osittain lasketuista gradienteista. Käytämme `alpha`-parametria määrittämään, missä määrin gradientit ja palkkiot otetaan huomioon - tätä kutsutaan vahvistusalgoritmin *oppimisnopeudeksi*.
1. Lopuksi koulutamme verkkoamme tiloilla ja odotetuilla toimilla, ja toistamme prosessin.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Nyt suoritetaan jakso renderöinnillä nähdäksemme tuloksen:


In [ ]:
_ = run_episode(render=True)

Toivottavasti huomaat, että tanko pysyy nyt melko hyvin tasapainossa!

## Actor-Critic-malli

Actor-Critic-malli on jatkokehitys policy gradient -menetelmästä, jossa rakennamme neuroverkon oppimaan sekä toimintapolitiikan että arvioidut palkkiot. Verkolla on kaksi ulostuloa (tai voit ajatella sen olevan kaksi erillistä verkkoa):
* **Actor** suosittelee, mitä toimintoa tulisi käyttää, antamalla meille tilan todennäköisyysjakauman, kuten policy gradient -mallissa.
* **Critic** arvioi, millainen palkkio näistä toimista voisi olla. Se palauttaa arvioidut kokonaispalkkiot tulevaisuudessa annetussa tilassa.

Määritellään tällainen malli:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Meidän täytyisi hieman muokata `discounted_rewards`- ja `run_episode`-funktioitamme:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Nyt suoritetaan pääasiallinen koulutussilmukka. Käytämme manuaalista verkon koulutusprosessia laskemalla oikeat häviöfunktiot ja päivittämällä verkon parametrit:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Yhteenveto

Tässä demossa olemme tutustuneet kahteen vahvistusoppimisalgoritmiin: yksinkertaiseen policy gradient -menetelmään ja kehittyneempään actor-critic -menetelmään. Voit huomata, että nämä algoritmit toimivat abstraktien tilan, toiminnan ja palkkion käsitteiden avulla – siksi niitä voidaan soveltaa hyvin erilaisiin ympäristöihin.

Vahvistusoppimisen avulla voimme oppia parhaan strategian ongelman ratkaisemiseksi pelkästään tarkastelemalla lopullista palkkiota. Se, että emme tarvitse valmiiksi merkittyjä aineistoja, mahdollistaa simulaatioiden toistamisen useita kertoja malliemme optimoimiseksi. Tästä huolimatta RL:ssä on yhä monia haasteita, joihin voit perehtyä tarkemmin, jos päätät syventyä tähän mielenkiintoiseen tekoälyn osa-alueeseen.



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäinen asiakirja sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
